Maintained by: Ros

### <b>PLS SEE </b>: <br> I wrote a class for chunking pipeline. Please scroll all the way down on sample usage

In [3]:
from langchain.agents import Tool
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
#from pydantic import BaseModel, Field
from pydantic.v1 import BaseModel, Field

# from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# Credentials
from dotenv import load_dotenv
from pathlib import Path
import os

# Llama parser
from llama_parse import LlamaParse  # pip install llama-parse
import nest_asyncio

# Regex 
import re 

#### Step 1. Use Llama-Parse to convert pdf into a markdown text which previous the structure of the document

In [ ]:
filepath = './data/Employment_Act_1968.pdf'

In [ ]:
# Use LlamaParse to convert the PDF into a markdown format

nest_asyncio.apply()

parser = LlamaParse(
    api_key="llx-ryQ68yiosrPCHZo5xWy91S8d5qVR5SoUMzkkBrW2LjVwnxum",  # you will need an API key, get it from https://cloud.llamaindex.ai/
    result_type="markdown"  # "markdown" and "text" are available
)

documents = parser.load_data(filepath)

Started parsing the file under job_id 8fdbf67f-ca1e-4cf2-bf82-f816511bc632


In [ ]:
type(documents[0])


llama_index.core.schema.Document

In [ ]:
employment_act_markdown = documents[0].get_content()

In [ ]:
# Save the parsed markdown document as a .txt file
def save_as_txt(text, filename):
    with open(filename, "w") as file:
        file.write(text)

# Example usage:
filename = "employment_act_markdown.txt"
save_as_txt(employment_act_markdown, filename)
print(f"Text saved to '{filename}' successfully.")


Text saved to 'employment_act_markdown.txt' successfully.


Processing the Markdown text
1. Remove the repetitive headers
2. Split using '##' as delimiters

In [ ]:
filename = "../employment_act_markdown.txt"

with open(filename, 'r') as file:
    employment_act = file.read()
    print(employment_act)

# THE STATUTES OF THE REPUBLIC OF SINGAPORE

## EMPLOYMENT ACT 1968

2020 REVISED EDITION

This revised edition incorporates all amendments up to and including 1 December 2021 and comes into operation on 31 December 2021.

Prepared and Published by

THE LAW REVISION COMMISSION

UNDER THE AUTHORITY OF

THE REVISED EDITION OF THE LAWS ACT 1983

Informal Consolidation – version in force from 29/5/2022
---
## 2020 Ed.

### Employment Act 1968

ARRANGEMENT OF SECTIONS
PART 1
PRELIMINARY
Section
1. Short title
2. Interpretation
3. Appointment of officers
4. Rules and orders
5. Minister may restrict application
6. Existing law not affected
7. [Repealed]
PART 2
CONTRACTS OF SERVICE
8. Illegal terms of contract of service
9. Termination of contract
10. Notice of termination of contract
11. Termination of contract wipout notice
12. Contractual age
13. When contract deemed to be broken by employer and employee
14. Dismissal
15. Termination by employee preatened by danger
16. Liability on breach o

#### 2. Using an excerpt of the employment act to try out chunking

Libraries

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Any
import re

In [ ]:
# Load text 
# sample_filename = '../sample_text.txt'
# loader = TextLoader(sample_filename)
# sample_doc = loader.load()

filename = "../sample_text2.txt"

with open(filename, 'r') as file:
    sample_text = file.read()

In [ ]:
print(sample_text)

## Dismissal

14.—(1) An employer may after due inquiry dismiss without notice an employee employed by the employer on the grounds of misconduct inconsistent with the fulfillment of the express or implied conditions of the employee’s service, except that instead of dismissing an employee an employer may —

- (a) instantly down-grade the employee; or
- (b) instantly suspend the employee from work without payment of salary for a period not exceeding one week.

[26/2013]

(2) Despite subsection (1), but subject to section 3 of the Employment Claims Act 2016, where a relevant employee considers that he or she has been dismissed without just cause or excuse by his or her employer, the employee may lodge a claim, under section 13 of that Act, for either of the following remedies:

- (a) reinstatement in the employee’s former employment;
- (b) compensation.

[55/2018]

(2A) For the purposes of subsection (2), a relevant employee means —

- (a) an employee employed in a managerial or an execut

In [ ]:
# Remove repetitive headers

'''Variations 
## Employment Act 1968 2020 Ed.
## 2020 Ed. Employment Act 1968
'''

sample_text_cleaned = re.sub(r'## Employment Act 1968 2020 Ed.', '', sample_text)
sample_text_cleaned = re.sub(r'## 2020 Ed. Employment Act 1968', '', sample_text_cleaned)

In [ ]:
print(sample_text_cleaned)

## Dismissal

14.—(1) An employer may after due inquiry dismiss without notice an employee employed by the employer on the grounds of misconduct inconsistent with the fulfillment of the express or implied conditions of the employee’s service, except that instead of dismissing an employee an employer may —

- (a) instantly down-grade the employee; or
- (b) instantly suspend the employee from work without payment of salary for a period not exceeding one week.

[26/2013]

(2) Despite subsection (1), but subject to section 3 of the Employment Claims Act 2016, where a relevant employee considers that he or she has been dismissed without just cause or excuse by his or her employer, the employee may lodge a claim, under section 13 of that Act, for either of the following remedies:

- (a) reinstatement in the employee’s former employment;
- (b) compensation.

[55/2018]

(2A) For the purposes of subsection (2), a relevant employee means —

- (a) an employee employed in a managerial or an execut

In [ ]:
# Referred to langchain's source code and documentation
# https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter
# https://github.com/langchain-ai/langchain/blob/master/libs/text-splitters/langchain_text_splitters/character.py

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    separators=['(?<=\n)##', '(?<=\n)###'],
    is_separator_regex=True,
)

In [ ]:
texts = text_splitter.create_documents([sample_text_cleaned])

In [ ]:
texts

[Document(page_content='## Dismissal\n\n14.—(1) An employer may after due inquiry dismiss without notice an employee employed by the employer on the grounds of misconduct inconsistent with the fulfillment of the express or implied conditions of the employee’s service, except that instead of dismissing an employee an employer may —\n\n- (a) instantly down-grade the employee; or\n- (b) instantly suspend the employee from work without payment of salary for a period not exceeding one week.\n\n[26/2013]\n\n(2) Despite subsection (1), but subject to section 3 of the Employment Claims Act 2016, where a relevant employee considers that he or she has been dismissed without just cause or excuse by his or her employer, the employee may lodge a claim, under section 13 of that Act, for either of the following remedies:\n\n- (a) reinstatement in the employee’s former employment;\n- (b) compensation.\n\n[55/2018]\n\n(2A) For the purposes of subsection (2), a relevant employee means —\n\n- (a) an empl

In [ ]:
print(texts[1].page_content)

## Termination by employee threatened by danger

15. An employee may terminate his or her contract of service with his or her employer without notice where the employee or the employee’s dependant is immediately threatened by danger to the person by violence or disease that the employee did not by his or her contract of service undertake to run.




In [ ]:
print(texts[2].page_content)

## Liability on breach of contract

16. Subject to anything in the contract of service to the contrary, the party who breaks the contract of service is liable to pay to the other party a sum equal to the amount the firstmentioned party would have been liable to pay under section 11 had the firstmentioned party terminated the contract of service without notice or with insufficient notice.




##### 3. Test the chunk_pipeline class

In [6]:
import os
import pickle

In [7]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src')  # Add the path to the 'src' folder
from chunking import ChunkPipeline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Instantiate class 
filename = '../data/employment_act_markdown.txt'
chunk_pipeline = ChunkPipeline(filename)

# Clean text
cleaned_text = chunk_pipeline.clean_text()

# Generate splits
all_splits = chunk_pipeline.splitter(cleaned_text)

In [9]:
len(all_splits)

121

In [10]:
all_splits[28:32]

[Document(page_content='## Contract of service not to restrict rights of employees to join, participate in or organise trade unions\n\n17. Subject to any other written law for the time being in force, nothing in any contract of service restricts, in any way, the right of any employee who is a party to such contract —\n\n- (a) to join a registered trade union;\n- (b) to participate in the activities of a registered trade union, whether as an officer of the trade union or otherwise; or\n- (c) to associate with any other persons for the purpose of organising a trade union in accordance with the provisions of the Trade Unions Act 1940.\n\nInformal Consolidation – version in force from 29/5/2022\n---\n\n\nChange of employer\n(1) If by or under any written law a contract of employment between any body corporate and an employee is modified and some oper body corporate is substituted as pe employer, pe employee’s period of employment at pe time when pe modification takes effect counts as a per

### Checking if some text is repeated in chunks

In [44]:
search_phrase = "Termination by employee threatened by danger"
# "Payment of salary through bank"
# "Termination by employee threatened by danger"

In [45]:
# Check for the phrase in each document
documents_with_phrase = [doc for doc in all_splits if search_phrase in doc.page_content]

In [46]:
# Count the documents containing the phrase
number_of_documents_with_phrase = len(documents_with_phrase)

number_of_documents_with_phrase

1

In [47]:
indexes_with_phrase = [index for index, doc in enumerate(all_splits) if search_phrase in doc.page_content]

indexes_with_phrase

[26]

In [48]:
all_splits[26]

Document(page_content='## Termination by employee threatened by danger\n\n15. An employee may terminate his or her contract of service with his or her employer without notice where the employee or the employee’s dependant is immediately threatened by danger to the person by violence or disease that the employee did not by his or her contract of service undertake to run.\n\n')

## Embedding

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

In [8]:
# Create the embedding function using your model of choice
embedding_function = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [9]:
# Initialize Chroma with the documents and embedding function
db = Chroma.from_documents(all_splits, embedding_function)

In [12]:
import pickle

In [12]:
with open('../data/all_splits.pkl', 'wb') as file:
    pickle.dump(all_splits, file)

In [13]:
with open('../data/embedding_function.pkl', 'wb') as file:
    pickle.dump(embedding_function, file)

In [14]:
# with open('../data/chroma_db.pkl', 'wb') as file:
#     pickle.dump(db, file)

In [36]:
# Create a retriever from vectorstore (db)
retriever = db.as_retriever(k=4)

docs = retriever.invoke("CWhat are the key provisions about annual leave")

docs

[Document(page_content='## Annual leave\n\n88A.—(1) An employee who has served an employer for a period of not less than 3 months is, in addition to the rest days, holidays and sick leave to which the employee is entitled under sections 36, 88 and 89, respectively, entitled to the following:\n\n(a) 7 days of paid annual leave, for pe first 12 monps of continuous service wip pe same employer;\n(b) subject to paragraph (c), an additional one day of paid annual leave, for every subsequent 12 monps of continuous service wip pe same employer;\n(c) a maximum of 14 days of paid annual leave. [55/2018]\n\n(2) An employee who has served an employer for a period of not less than 3 months, but has not completed 12 months of continuous service in any year, is entitled to annual leave in proportion to the number of completed months of service in that year. [55/2018]\n\n(3) In calculating the proportionate annual leave under subsection (2) —\n\n- (a) any fraction of a day that is less than one-half 

In [ ]:
# Ignore test code below

In [31]:
# Example query about the Employment Act
query = "What are the key provisions about annual leave?"

# Queries the database
docs = db.similarity_search(query)

# Print results
for doc in docs:
    print(doc.page_content)

## Annual leave

88A.—(1) An employee who has served an employer for a period of not less than 3 months is, in addition to the rest days, holidays and sick leave to which the employee is entitled under sections 36, 88 and 89, respectively, entitled to the following:

(a) 7 days of paid annual leave, for pe first 12 monps of continuous service wip pe same employer;
(b) subject to paragraph (c), an additional one day of paid annual leave, for every subsequent 12 monps of continuous service wip pe same employer;
(c) a maximum of 14 days of paid annual leave. [55/2018]

(2) An employee who has served an employer for a period of not less than 3 months, but has not completed 12 months of continuous service in any year, is entitled to annual leave in proportion to the number of completed months of service in that year. [55/2018]

(3) In calculating the proportionate annual leave under subsection (2) —

- (a) any fraction of a day that is less than one-half of a day must be disregarded;

Infor

In [40]:
# Extract text from each Document object in all_splits
text_chunks = [doc.page_content for doc in all_splits]

In [43]:
def query(text_batch):
    response = requests.post(api_url, headers=headers, json={"inputs": text_batch})
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Query failed with status code {response.status_code}")

In [44]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = os.environ["hf_token"]

In [45]:
import requests

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}


In [46]:
def query_batched(text_chunks, batch_size=10):
    embeddings = []
    for i in range(0, len(text_chunks), batch_size):
        batch = text_chunks[i:i + batch_size]
        batch_embeddings = query(batch)
        embeddings.extend(batch_embeddings)
    return embeddings

In [47]:
# Example usage
batch_size = 10  # Adjust based on the size of your texts and what the API can handle
all_embeddings = query_batched(text_chunks, batch_size=batch_size)

In [48]:
import json

# Save embeddings to JSON file
with open("embeddings.json", "w") as f:
    json.dump(all_embeddings, f)

In [49]:
with open("embeddings.json", "r") as f:
    loaded_embeddings = json.load(f)

In [51]:
len(loaded_embeddings)

121

In [52]:
len(text_chunks)

121

In [14]:
# def query(texts):
#     response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
#     return response.json()

In [15]:
# texts = [
#     "How many days of paid leave should I get?",
#     "Will I get retrenchment benefits if I am retrenched?"
# ]

In [18]:
# qn_embeddings = query(texts)